In [23]:
from pm4py.objects.log.importer.xes import importer as xes_importer

In [2]:
filepath = "data/"

log = xes_importer.apply(filepath + 'DomesticDeclarations.xes')

HBox(children=(FloatProgress(value=0.0, description='parsing log, completed traces :: ', max=10500.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

### Example "Domestic Declarations" dataset from the BPI Challenge 2020:
* contains events pertaining to two years of travel expense claims
* contains 10,500 cases, 56,437 events
* process flow: 
    * after submission by the employee, the request is sent for approval to the travel administration
    * if approved, the request is then forwarded to the budget owner and after that to the supervisor
    * if the budget owner and supervisor are the same person, then only one of these steps is taken
    * in some cases, the director also needs to approve the request
    * process finished with either the trip taking place or a payment being requested and payed.
* for domestic trips, no prior permission is needed, i.e. an employee can undertake these trips and ask for reimbursement of the costs afterwards
* to get the costs for a travel reimbursed, a claim is filed. This can be done as soon as costs are actually payed (for example for flights or conference registration fees), or within two months after the trip (for example hotel and food costs which are usually payed on the spot)

In [30]:
print("first trace of the log: \n", log[0], "\n")
print("first event of the first trace: \n", log[0][0], "\n")

first trace of the log: 
 {'attributes': {'id': 'declaration 86791', 'BudgetNumber': 'budget 86566', 'concept:name': 'declaration 86791', 'DeclarationNumber': 'declaration number 86792', 'Amount': 26.85120450862128}, 'events': [{'id': 'st_step 86794_0', 'org:role': 'EMPLOYEE', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'org:resource': 'STAFF MEMBER'}, '..', {'id': 'dd_declaration 86791_20', 'org:role': 'UNDEFINED', 'concept:name': 'Payment Handled', 'time:timestamp': datetime.datetime(2017, 1, 12, 17, 31, 22, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'org:resource': 'SYSTEM'}]} 

first event of the first trace: 
 {'id': 'st_step 86794_0', 'org:role': 'EMPLOYEE', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'org:resource': 'STAFF

In [17]:
type(log)

pm4py.objects.log.log.EventLog

Event Log (objects.log.log.EventLog): 
Represents a sequence of sequences of events. The concept of an event log is the more traditional view on event data, i.e., executions of a process are captured in traces of events.